# ChargeUp! Battery BMS Data Analysis (Public)

This notebook provides a framework for the analysis of Battery BMS data used with e-motorcycles and was developed as part of the **ChargeUp!** project (2022-2023), funded by **P4G** (https://p4gpartnerships.org/chargeup). 

Author: Cameron Sheehan (Research Associate, Energy Futures Lab, Imperial College London)

## 1. Import all required packages

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from keplergl import KeplerGl
import random
from h3 import h3
import h3pandas
from folium import Map, Marker, GeoJson
from folium.plugins import MarkerCluster
import branca.colormap as cm
from branca.colormap import linear
import folium
import networkx as nx
import osmnx as ox
import shapely
from shapely.geometry import LineString
from shapely.geometry import Point
from shapely.geometry import Polygon
import pulp
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable
from descartes import PolygonPatch
from rasterstats import zonal_stats
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import normalize
import movingpandas as mpd
import xarray as xr
import hvplot.xarray  # noqa
import hvplot.pandas 
from holoviews import opts
import seaborn as sns
from pytz import common_timezones, all_timezones
import warnings
from IPython.display import display, HTML

display(HTML("<style>.output_result { max-width:100% !important; }</style>"))

plt.rcParams['axes.axisbelow'] = True

warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

## 2. Import Data

Open files as dataframes and process column data.

First, paste full pathname to the data files below, ensure pathname is inside inverted commas, i.e. '/XXX/XXX/XXX/battery-bms.csv'

In [ ]:
# Set data file pathnames
pathname_battery_gps = ''
pathname_battery_bms = ''
pathname_BSS_loc = ''

Next, set the column headings for the required data fields:

In [ ]:
# For BMS and GPS datasets
batt_id_col = '' 

# For GPS dataset
gps_timestamp_col = ''
gps_lat_col = ''
gps_lng_col = ''

# For BMS dataset
bms_timestamp_col = ''
soc_col = ''

# For BSS locations dataset
BSS_lat_col = ''
BSS_lng_col = ''
BSS_id_col = '';
BSS_loc_name_col = ''

Read data from csv/parquet files and create pandas dataframes: <br>

Note: Uncomment the correct option depending on file types (csv/parquet)

In [ ]:
# df_battery_gps_raw = pd.read_parquet(pathname_battery_gps) 
df_battery_gps_raw = pd.read_csv(pathname_battery_gps, header=0) 
df_battery_gps = pd.DataFrame()

df_battery_bms_raw = pd.read_parquet(pathname_battery_bms)
# df_battery_bms_raw = pd.read_csv(pathname_battery_bms, header=0) 
df_battery_bms = pd.DataFrame()

# df_BSS_loc_raw = pd.read_parquet(pathname_BSS_loc)
df_BSS_loc_raw = pd.read_csv(pathname_BSS_loc, header=0)
df_BSS_loc = pd.DataFrame()

Assign data columns to correct dataframe column headings for required data fields:

In [ ]:
df_battery_gps[['gps_timestamp','batt_id','gps_lat','gps_lng']] = df_battery_gps_raw[[gps_timestamp_col,batt_id_col,gps_lat_col,gps_lng_col]]

df_battery_bms[['bms_timestamp','batt_id','soc']] = df_battery_bms_raw[[bms_timestamp_col,batt_id_col,soc_col]]

df_BSS_loc[['BSS_id','BSS_loc_name','BSS_lat','BSS_lng']] = df_BSS_loc_raw[[BSS_id_col,BSS_loc_name_col,BSS_lat_col,BSS_lng_col]]

#### Process data types and required timezones:

Note: To check available timezone names, uncomment and run the cell below.

In [ ]:
# all_timezones

In [ ]:
# Process battery_gps data
df_battery_gps['batt_id'] = df_battery_gps['batt_id'].astype("string")
df_battery_gps["gps_time"] = pd.to_datetime(df_battery_gps['gps_timestamp'], unit='s', utc=True).dt.tz_convert('Africa/Nairobi')
df_battery_gps = df_battery_gps.sort_values(by=['batt_id','gps_time'])
df_battery_gps = df_battery_gps.set_index('gps_time')
# df_battery_gps = df_battery_gps.drop(columns=['Time','cmd','date','speed','course','altitude'])

# Process battery_bms data
df_battery_bms["Time"] = pd.to_datetime(df_battery_bms['bms_timestamp'], unit='s', utc=True).dt.tz_convert('Africa/Nairobi')
df_battery_bms['batt_id'] = df_battery_bms['batt_id'].astype("string")
df_battery_bms = df_battery_bms.sort_values(by=['batt_id','Time'])
df_battery_bms = df_battery_bms.dropna(subset=['Time'])

# Process BSS_loc data
gdf_BSS_loc = gpd.GeoDataFrame(
    df_BSS_loc, geometry=gpd.points_from_xy(df_BSS_loc.BSS_lng, df_BSS_loc.BSS_lat, crs="EPSG:4326"), crs="EPSG:4326")

## 3. Analyse BMS data

Check number of unique devices and data entries in dataset:

In [ ]:
n_batteries = len(df_battery_bms.batt_id.unique())
print("There are a total of ", n_batteries, " unique batteries, and ", len(df_battery_bms.index), " data entries in this BMS dataset")

#### Inspect SOC values of all batteries between two dates using a plot:

In [ ]:
# Select DataFrame rows between two dates - set start and end dates
start_date = '2023-01-16'
end_date = '2023-01-18'
mask = (df_battery_bms.Time >= start_date) & (df_battery_bms.Time <= end_date)
df_battery_bms_t_range = df_battery_bms.loc[mask]

In [ ]:
ax = sns.relplot(data=df_battery_bms_t_range, hue='batt_id', x='Time', y='soc', 
                kind='line', height=3, aspect=3, alpha=0.6)
ax.fig.autofmt_xdate()

#### Add time increments for various analyses 

Note: this method ignores seconds when categorising time increments

In [ ]:
# Hourly increments
df_battery_bms['hr_inc'] = df_battery_bms['Time'].dt.hour + np.ceil(df_battery_bms['Time'].dt.minute/60)
df_battery_bms.loc[df_battery_bms['hr_inc']==0, 'hr_inc'] = 24
df_battery_bms['hr_inc'] = df_battery_bms['hr_inc'].astype('int')

# 30 minute increments
df_battery_bms['30_mins_inc'] = df_battery_bms['Time'].dt.hour + np.ceil(df_battery_bms['Time'].dt.minute/30)*30/60
df_battery_bms.loc[df_battery_bms['30_mins_inc']==0, '30_mins_inc'] = 24

# 15 minute increments
df_battery_bms['15_mins_inc'] = df_battery_bms['Time'].dt.hour + np.ceil(df_battery_bms['Time'].dt.minute/15)*15/60
df_battery_bms.loc[df_battery_bms['15_mins_inc']==0, '15_mins_inc'] = 24

Check resulting time increments by viewing the dataframe

In [ ]:
df_battery_bms.head()

## 4. Calculate energy usage based on battery SOC changes

First, set the capacity of the batteries that were in the dataset:

In [ ]:
batt_cap =  # battery capacity in kWh

Then, calculate energy changes and classify different entries as charging:

In [ ]:
# Initialise 'charging' column values to be zero
df_battery_bms['charging'] = 0.0
# Calculate change in SOC values of each battery between successive time entries
df_battery_bms['soc_change'] = df_battery_bms.groupby(['batt_id'])['soc'].diff().fillna(0)
# Calculate energy change based on SOC change and battery capacity i.e. energy change (kWh) = SOC (%) x Battery Cap. (kWh)
df_battery_bms['energy_change'] = df_battery_bms['soc_change']/100*batt_cap
# Calculate time difference between successive time entries for each battery
df_battery_bms['t_diff'] = df_battery_bms.groupby(['batt_id'])['Time'].diff().dt.seconds.div(60, fill_value=0)
# Classify battery as charging (i.e. 'charging' = 1) if SOC change is positive
df_battery_bms.loc[df_battery_bms['soc_change']>0, 'charging'] = 1
# Remove entries where SOC change is greater than 20% in less than 5 mins (very fast change unlikely) - values can be adjusted
df_battery_bms = df_battery_bms.drop(df_battery_bms[(abs(df_battery_bms['soc_change'])>20) & (df_battery_bms['t_diff']<=5)].index)
# Create subset of battery_bms data for all entries classified as charging / in use
df_battery_bms_ch = df_battery_bms[df_battery_bms['charging']==1]
df_battery_bms_use = df_battery_bms[df_battery_bms['charging']==0]
# Determine the total number of days where charging occurred
total_ch_days = df_battery_bms_ch['Time'].dt.date.nunique()
# Determine the total number of batteries that were charged at some point within the dataset 
total_ch_devices = df_battery_bms_ch['batt_id'].nunique()

# Display the updated battery bms dataframe
df_battery_bms.head()

In [ ]:
# Display the charging subset of battery bms dataframe
df_battery_bms_ch.head()

#### Calculate total energy values

In [ ]:
total_ch_energy = df_battery_bms_ch['energy_change'].sum()
print('The total amount of energy used for charging during the analysis period  was: \n', np.round(total_ch_energy, decimals = 2),' kWh' )

In [ ]:
total_used_energy = -df_battery_bms_use['energy_change'].sum()
print('The total amount of energy used by the e-motorycles during the analysis period  was: \n', np.round(total_used_energy, decimals = 2),' kWh' )

#### Calculate average Energy Intensity (kWh/km) and estimated range of e-motorcycles in dataset

In [ ]:
# Input total distance of vehicles in dataset using the value from the GPS data analysis:
total_distance_all_vehicles =  

# Caluclate the average energy intensity
energy_intensity_avg = total_used_energy/total_distance_all_vehicles

print('The average energy intensity of the e-motorycles during the analysis period  was: \n', np.round(energy_intensity_avg, decimals = 3),' kWh/km' )

In [ ]:
estimated_battery_range = batt_cap/energy_intensity_avg

print('The estimated range of the batteries used with the e-motorycles during the analysis period  was: \n', np.round(estimated_battery_range, decimals = 2),' km' )

## 5. Determine where charging events occurred

#### Merge BMS and GPS datasets into a single dataframe

In [ ]:
df_battery_gps=df_battery_gps.reset_index()

# Create a 'Time' column that both datasets could be merged on 
df_battery_gps['Time'] = df_battery_gps['gps_time']
df_battery_bms['Time_bms'] = df_battery_bms['Time']

# Sort dataframes by time
df_battery_gps_s = df_battery_gps.sort_values(['Time'])
df_battery_bms_s = df_battery_bms.sort_values(['Time'])

# Merge the BMS and GPS datsets using the battery ID's and the closest times of the datapoints (max 30 seconds difference, value can be changed).
df_merged_bms_gps = pd.merge_asof(df_battery_gps_s,df_battery_bms_s, on="Time", by='batt_id', direction='nearest', suffixes=('_gps', '_bms'), tolerance=pd.Timedelta(seconds=30))
# Drop rows that contain an NA value in the soc column
df_merged_bms_gps = df_merged_bms_gps.dropna(subset=['soc'])
# Drop duplicates where the same BMS datapoint was assigned to more than one GPS datapoint (keeps first value)
df_merged_bms_gps = df_merged_bms_gps.drop_duplicates(subset=['Time_bms','batt_id'])

# Display merged dataset
df_merged_bms_gps.head()

#### Recalculate SOC and Time difference values between each entry since the number of entries changed after merging

In [ ]:
# Initialise 'charging' column values to be zero
df_merged_bms_gps['charging'] = 0
# Calculate change in SOC values of each battery between successive time entries
df_merged_bms_gps['soc_change'] = df_merged_bms_gps.groupby(['batt_id'])['soc'].diff().fillna(0)
# Calculate energy change based on SOC change and battery capacity i.e. energy change (kWh) = SOC (%) x Battery Cap. (kWh)
df_merged_bms_gps['energy_change'] = df_merged_bms_gps['soc_change']/100*batt_cap
# Calculate time difference between successive time entries for each battery
df_merged_bms_gps['t_diff'] = df_merged_bms_gps.groupby(['batt_id'])['Time_bms'].diff().dt.seconds.div(60, fill_value=0)
# Classify battery as charging (i.e. 'charging' = 1) if SOC change is positive
df_merged_bms_gps.loc[df_merged_bms_gps['soc_change']>0, 'charging'] = 1
# Remove entries where SOC change is greater than 20% in less than 5 mins (very fast change unlikely) - values can be adjusted
df_merged_bms_gps = df_merged_bms_gps.drop(df_merged_bms_gps[(abs(df_merged_bms_gps['soc_change'])>20) & (df_merged_bms_gps['t_diff']<=5)].index)
# Create subset of battery_bms data for all entries classified as charging
df_merged_bms_gps_ch = df_merged_bms_gps[df_merged_bms_gps['charging']==1]

#### Convert dataframes to geodataframes

In [ ]:
gdf_merged_bms_gps = gpd.GeoDataFrame(
    df_merged_bms_gps, geometry=gpd.points_from_xy(df_merged_bms_gps.gps_lng, df_merged_bms_gps.gps_lat, crs="EPSG:4326"), crs="EPSG:4326")

gdf_merged_bms_gps_ch = gpd.GeoDataFrame(
    df_merged_bms_gps_ch, geometry=gpd.points_from_xy(df_merged_bms_gps_ch.gps_lng, df_merged_bms_gps_ch.gps_lat, crs="EPSG:4326"), crs="EPSG:4326")

#### Add search region (buffer) around existing BSS locations to use for allocating charging GPS points to BSS location

In [ ]:
gdf_BSS_loc['geometry'] = gdf_BSS_loc.buffer(0.002)
gdf_BSS_loc_uniq = gdf_BSS_loc.drop_duplicates(['geometry'])

Determine which charging events occurred at the existing BSSs

In [ ]:
gdf_merged_bms_gps_ch_at_bss = gdf_merged_bms_gps_ch.sjoin(gdf_BSS_loc_uniq, how="inner", predicate="within")
gdf_merged_bms_gps_ch_at_bss['hr_inc'] = gdf_merged_bms_gps_ch_at_bss['hr_inc'].astype('int')

Determine which charging events occurred away from existing BSSs (home/private charging)

In [ ]:
gdf_merged_bms_gps_ch_at_home = gdf_merged_bms_gps_ch.drop(gdf_merged_bms_gps_ch_at_bss.index)
gdf_merged_bms_gps_ch_at_home['hr_inc'] = gdf_merged_bms_gps_ch_at_home['hr_inc'].astype('int')

### Visualise charging location data

#### *Important Data Protection Note: This map contains potentially sensitive location data of individual trips and stops. This map should never be published publicly without the explicit consent of all individuals whose battery GPS data was included in the analysed dataset.*

In [ ]:
m_ch_loc = KeplerGl(height=800)
# Add gps records
m_ch_loc.add_data(gdf_merged_bms_gps_ch.copy(), 'All merged charging points')
m_ch_loc.add_data(gdf_BSS_loc.copy(), 'Existing BSS sites')
m_ch_loc.add_data(gdf_merged_bms_gps_ch_at_bss.copy(), 'Charging points within BSS rings')
m_ch_loc.add_data(gdf_merged_bms_gps_ch_at_home.copy(), 'Charging points at home')
%run 'Kepler configs/map_config_charging_loc.py'
m_ch_loc.config = config

In [ ]:
m_ch_loc

In [ ]:
# with open('Kepler configs/map_config_charging_loc.py', 'w') as f:
#    f.write('config = {}'.format(m_ch_loc.config))

## 6. Create bar plots of the aggregated charging energy over entire analysis period


#### Set charging efficiency to determine amount of actual energy used to charge battery

In [ ]:
charging_efficiency = ;

#### All charging energy usage aggregated for every hour of day

In [ ]:
df_battery_bms_ch_energy_hr = df_battery_bms_ch.groupby('hr_inc').energy_change.agg('sum').to_frame('tot_ch_energy')/charging_efficiency
ax = df_battery_bms_ch_energy_hr.plot.bar(figsize=(8,3), color="blue", width=0.8, rot=45, legend=False)
# ax.set_title('All charging energy usage aggregated for every hour of day')
ax.set_title('')
ax.set_xlabel("Time period (hours)");
ax.set_ylabel("Total Charging Energy (kWh)");

In [ ]:
# ax.figure.savefig("images/Charging energy usage aggregated for every hour of day.png", bbox_inches='tight')

#### All charging energy usage aggregated for every 30 mins of day

In [ ]:
df_battery_bms_ch_energy_30_mins = df_battery_bms_ch.groupby('30_mins_inc').energy_change.agg('sum').to_frame('tot_ch_energy')/charging_efficiency
ax = df_battery_bms_ch_energy_30_mins.plot.bar(figsize=(10,3), color="blue", width=0.8, rot=90, legend=False)
# ax.set_title('Battery swap station charging energy usage aggregated for every 30 mins of day')
ax.set_title('')
ax.set_xlabel("Time period (hours)");
ax.set_ylabel("Total Charging Energy (kWh)");

In [ ]:
# ax.figure.savefig("images/Charging energy usage aggregated for every 30 mins of day.png", bbox_inches='tight')

#### BSS charging energy usage aggregated for every hour of day

In [ ]:
df_merged_bms_gps_ch_energy_at_bss_hr = gdf_merged_bms_gps_ch_at_bss.groupby('hr_inc').energy_change.agg('sum').to_frame('tot_ch_energy')/charging_efficiency
ax = df_merged_bms_gps_ch_energy_at_bss_hr.plot.bar(figsize=(8,3), color="blue", width=0.8, rot=45, legend=False)
# ax.set_title('Battery swap station charging energy usage aggregated for every hour of day')
ax.set_title('')
ax.set_xlabel("Time (hours)");
ax.set_ylabel("Total Charging Energy (kWh)");

In [ ]:
# ax.figure.savefig("images/BSS charging energy usage aggregated for every hour of day.png", bbox_inches='tight')

#### BSS charging energy usage aggregated for every half hour of day

In [ ]:
df_merged_bms_gps_ch_energy_at_bss_30_mins = gdf_merged_bms_gps_ch_at_bss.groupby('30_mins_inc').energy_change.agg('sum').to_frame('tot_ch_energy')/charging_efficiency
ax = df_merged_bms_gps_ch_energy_at_bss_30_mins.plot.bar(figsize=(10,3), color="blue", width=0.8, rot=90, legend=False)
# ax.set_title('Battery swap station charging energy usage aggregated for every hour of day')
ax.set_title('')
ax.set_xlabel("Time (hours)");
ax.set_ylabel("Total Charging Energy (kWh)");

In [ ]:
# ax.figure.savefig("images/BSS charging energy usage aggregated for every half hour of day.png", bbox_inches='tight')

#### Private charging energy usage aggregated for every hour of day

In [ ]:
df_merged_bms_gps_ch_energy_at_home_hr = gdf_merged_bms_gps_ch_at_home.groupby('hr_inc').energy_change.agg('sum').to_frame('tot_ch_energy')/charging_efficiency
ax = df_merged_bms_gps_ch_energy_at_home_hr.plot.bar(figsize=(8,3), color="blue", width=0.8, rot=45, legend=False)
# ax.set_title('Private charging energy usage aggregated for every hour of day')
ax.set_title('')
ax.set_xlabel("Time (hours)");
ax.set_ylabel("Total Charging Energy (kWh)");

In [ ]:
# ax.figure.savefig("images/Private charging energy usage aggregated for every hour of day.png", bbox_inches='tight')

#### Charging energy usage aggregated for every hour of day for different locations

Create a single dataframe with categorised charging to use for plotting.

In [ ]:
df_merged_bms_gps_ch_energy_locations = df_merged_bms_gps_ch.copy()
df_merged_bms_gps_ch_energy_locations['location'] = ''
df_merged_bms_gps_ch_energy_locations.loc[gdf_merged_bms_gps_ch_at_bss.index.to_list(),'location'] = 'BSS'
df_merged_bms_gps_ch_energy_locations.loc[gdf_merged_bms_gps_ch_at_home.index.to_list(),'location'] = 'home'
df_merged_bms_gps_ch_energy_locations['hr_inc'] = df_merged_bms_gps_ch_energy_locations['hr_inc'].astype('int')
# df_merged_bms_gps_ch_energy_locations.iloc[[gdf_merged_bms_gps_ch_at_bss.index.to_list()], 'location'] = 'BSS'

In [ ]:
df_merged_bms_gps_ch_energy_locations_hr_loc = df_merged_bms_gps_ch_energy_locations.groupby(['hr_inc','location']).energy_change.agg('sum').to_frame('tot_ch_energy')/charging_efficiency
ax = df_merged_bms_gps_ch_energy_locations_hr_loc.unstack().plot.bar(figsize=(8,4), width=0.8, rot=45, stacked=False, legend=True, color=['blue','lightblue'])
# ax.set_title('Battery swap station charging energy usage aggregated for every hour of day')
ax.set_title('');
ax.set_xlabel("Time period (hours)");
ax.set_ylabel("Total Charging Energy (kWh)");
plt.legend(['Charging at BSS','Charging away from BSS'], loc='upper left');

In [ ]:
# ax.figure.savefig("images/Charging energy usage aggregated for every hour of day for different locations.png", bbox_inches='tight')

#### BSS charging energy usage aggregated for every hour of day split by BSS location

In [ ]:
gdf_merged_bms_gps_ch_at_bss_hr = gdf_merged_bms_gps_ch_at_bss.groupby(['hr_inc','BSS_loc_name']).energy_change.agg('sum').to_frame('Total Charging Energy (kWh)').unstack()/charging_efficiency
ax = gdf_merged_bms_gps_ch_at_bss_hr.plot.bar(figsize=(20,8),width=0.8,stacked=False, legend=True)
# ax.set_title('Battery swap station charging energy usage aggregated for every hour of day (charging within BSS locations only)')
ax.set_title('');
ax.set_xlabel("Time (hours)");
ax.set_ylabel("Total Charging Energy (kWh)");

## 7. Investigate how much savings could be achieved by shifting energy use away from the evening peak using co-ordinated charging at BSSs

### Determine the amount of energy  to be shifted during co-ordinated charging 

In [ ]:
# Set the different hours for each period required to analyse evening peak energy shifting
peak_load_hrs = [19,20,21,22]
offpeak_load_shift_hrs = [23,24,1,2,3,4,5,6]

# Set the different half-hours for each period required to analyse evening peak energy shifting
peak_load_30_mins = [18.5,19,19.5,20,20.5,21,21.5,22]
offpeak_load_shift_30_mins = [22.5,23,23.5,24,0.5,1,1.5,2,2.5,3,3.5,4,4.5,5,5.5,6]

In [ ]:
# Calculate the total amount of energy that would need to shift if the energy during the evening peak period was reduced to the afternoon average.
total_energy_shift_hr = df_merged_bms_gps_ch_energy_at_bss_hr.loc[peak_load_hrs].sum()
total_energy_shift_30_mins = df_merged_bms_gps_ch_energy_at_bss_30_mins.loc[peak_load_30_mins].sum()
# Divide the total shifted energy evenly between the remaining offpeak hours (i.e. the additonal shifted energy for each off-peak hour)
total_energy_shift_spread_hr = total_energy_shift_hr/len(offpeak_load_shift_hrs)
total_energy_shift_spread_30_mins = total_energy_shift_30_mins/len(offpeak_load_shift_30_mins)

### Create the co-ordinated charging energy profiles for BSSs

In [ ]:
# Create the "shifted" energy profile for hourly periods
df_merged_bms_gps_ch_energy_at_bss_hr['tot_ch_energy_coord'] = df_merged_bms_gps_ch_energy_at_bss_hr['tot_ch_energy']
df_merged_bms_gps_ch_energy_at_bss_hr.loc[peak_load_hrs, 'tot_ch_energy_coord'] = 0
df_merged_bms_gps_ch_energy_at_bss_hr.loc[offpeak_load_shift_hrs, 'tot_ch_energy_coord'] = df_merged_bms_gps_ch_energy_at_bss_hr.loc[offpeak_load_shift_hrs, 'tot_ch_energy_coord'] + total_energy_shift_spread_hr[0]
df_merged_bms_gps_ch_energy_at_bss_hr['energy_change'] =  df_merged_bms_gps_ch_energy_at_bss_hr['tot_ch_energy_coord'] - df_merged_bms_gps_ch_energy_at_bss_hr['tot_ch_energy']

# Display the resulting dataframe
df_merged_bms_gps_ch_energy_at_bss_hr.head()

In [ ]:
# Create the "shifted" energy profile for half-hourly periods
df_merged_bms_gps_ch_energy_at_bss_30_mins['tot_ch_energy_coord'] = df_merged_bms_gps_ch_energy_at_bss_30_mins['tot_ch_energy']
df_merged_bms_gps_ch_energy_at_bss_30_mins.loc[peak_load_30_mins, 'tot_ch_energy_coord'] = 0
df_merged_bms_gps_ch_energy_at_bss_30_mins.loc[offpeak_load_shift_30_mins, 'tot_ch_energy_coord'] = df_merged_bms_gps_ch_energy_at_bss_30_mins.loc[offpeak_load_shift_30_mins, 'tot_ch_energy_coord'] + total_energy_shift_spread_30_mins[0]
df_merged_bms_gps_ch_energy_at_bss_30_mins['energy_change'] =  df_merged_bms_gps_ch_energy_at_bss_30_mins['tot_ch_energy_coord'] - df_merged_bms_gps_ch_energy_at_bss_30_mins['tot_ch_energy']

# Display the resulting dataframe
df_merged_bms_gps_ch_energy_at_bss_30_mins.head()

#### Create bar chart to compare the original and shifted hourly charging energy profiles

In [ ]:
ax = df_merged_bms_gps_ch_energy_at_bss_hr.plot.bar(figsize=(20,8), legend=True, width=0.9, color=['blue','lightblue', 'red']); #['blue','lightblue', 'red']
# ax.set_title('Battery swap station charging energy usage aggregated for every hour of day');
ax.set_title('');
ax.set_xlabel("Time (hours)");
ax.set_ylabel("Total Charging Energy (kWh)");
plt.legend(['Existing BSS charging energy profile','Co-ordinated BSS charging energy profile','Amount of energy shifted']);
ax.yaxis.grid(color='gray', linestyle='--')

In [ ]:
# ax.figure.savefig("images/BSS charging energy usage aggregated for every hour of day (co-ordinated charging 3)'.png", bbox_inches='tight')

#### Create bar chart to compare the original and shifted half-hourly charging energy profiles

In [ ]:
ax = df_merged_bms_gps_ch_energy_at_bss_30_mins.plot.bar(y=['tot_ch_energy','tot_ch_energy_coord','energy_change'], figsize=(20,8), legend=True, width=0.8)
# ax.set_title('Battery swap station charging energy usage aggregated for every half-hour of day')
ax.set_title('');
ax.set_xlabel("Time (hours)");
ax.set_ylabel("Total Charging Energy (kWh)");
plt.legend(['Normal charging energy profile','Shifted charging energy profile','Amount of energy shifted']);

## 8. Emissions analysis

### Kenyan Average Emissions Factor every 30 mins of day

In [ ]:
# Set csv data file pathnames
pathname_hi_therm_AEF_30_mins = ''
pathname_hi_renew_AEF_30_mins = ''

In [ ]:
# Read data from csv files and create pandas dataframes
df_hi_therm_AEF_30_mins = pd.read_excel(pathname_hi_therm_AEF_30_mins, sheet_name='EF_half_hr', header=0)
df_hi_renew_AEF_30_mins = pd.read_excel(pathname_hi_renew_AEF_30_mins, sheet_name='EF_half_hr', header=0)

#### Create plot to compare AEF scenarios 

In [ ]:
xticks=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]; #xticks=[0,3,6,9,12,15,18,21,24]
ax = df_hi_therm_AEF_30_mins.plot(x='Hour', y='AEF_30_mins', color='black', xlim=[0,24],ylim=[0,300], xticks=xticks, figsize=(12,4), legend=True) 
df_hi_renew_AEF_30_mins.plot(ax=ax, x='Hour', y='AEF_30_mins', color='blue')
# ax.set_title('Kenyan Average Emissions Factor every 30 mins of day')
ax.set_title('');
ax.set_xlabel("Time (hours)");
ax.set_ylabel("Average Emissions Factor (kgCO2/MWh)");
plt.legend(['High Thermal Scenario','High Renewables Scenario']);
# ax.yaxis.grid(color='gray', linestyle='--')

In [ ]:
# ax.figure.savefig("images/Kenyan Average Emissions Factor every 30 mins of day.png", bbox_inches='tight')

### Calculate charging emissions

#### Emissions for all charging (at BSSs and homes)

In [ ]:
df_battery_bms_ch_energy_30_mins = df_battery_bms_ch_energy_30_mins.reset_index()
df_battery_bms_ch_energy_30_mins['emissions_30_mins_hi_therm'] = df_battery_bms_ch_energy_30_mins['tot_ch_energy']/1000*df_hi_therm_AEF_30_mins['AEF_30_mins']
df_battery_bms_ch_energy_30_mins['emissions_30_mins_hi_renew'] = df_battery_bms_ch_energy_30_mins['tot_ch_energy']/1000*df_hi_renew_AEF_30_mins['AEF_30_mins']
df_battery_bms_ch_energy_30_mins.head()

#### Create bar chart to compare emission scenarios

In [ ]:
ax = df_battery_bms_ch_energy_30_mins.plot.bar(x='30_mins_inc', 
                                              y=['emissions_30_mins_hi_therm','emissions_30_mins_hi_renew'],
                                              ylim=[0,10],
                                              figsize=(12,3), 
                                              width=0.8,
                                              color={'emissions_30_mins_hi_therm': "grey", 'emissions_30_mins_hi_renew': "blue"},
                                              legend=True)
# ax.set_title('Charging emissions aggregated for every 30 mins of day');
ax.set_title('');
ax.set_xlabel("Time period (hours)");
ax.set_ylabel("Emissions (kgCO2)");
plt.legend(['High Thermal Scenario','High Renewables Scenario'], loc='upper left');


In [ ]:
# ax.figure.savefig("images/Charging emissions aggregated for every 30 mins of day.png", bbox_inches='tight')

#### Emissions for charging at BSSs ONLY

In [ ]:
# Calculate emissions using energy and AEF for each 30 min period
df_merged_bms_gps_ch_energy_at_bss_30_mins = df_merged_bms_gps_ch_energy_at_bss_30_mins.reset_index()
df_merged_bms_gps_ch_energy_at_bss_30_mins['emissions_30_mins_hi_therm'] = df_merged_bms_gps_ch_energy_at_bss_30_mins['tot_ch_energy']/1000*df_hi_therm_AEF_30_mins['AEF_30_mins']
df_merged_bms_gps_ch_energy_at_bss_30_mins['emissions_30_mins_hi_renew'] = df_merged_bms_gps_ch_energy_at_bss_30_mins['tot_ch_energy']/1000*df_hi_renew_AEF_30_mins['AEF_30_mins']
df_merged_bms_gps_ch_energy_at_bss_30_mins.head()

#### Create bar chart to compare emission scenarios

In [ ]:
ax = df_merged_bms_gps_ch_energy_at_bss_30_mins.plot.bar(x='30_mins_inc', 
                                              y=['emissions_30_mins_hi_therm','emissions_30_mins_hi_renew'],
                                              ylim=[0,4],
                                              figsize=(12,3), 
                                              width=0.8,
                                              color={'emissions_30_mins_hi_therm': "grey", 'emissions_30_mins_hi_renew': "blue"},
                                              legend=True)
# ax.set_title('Charging emissions aggregated for every 30 mins of day');
ax.set_title('');
ax.set_xlabel("Time period (hours)");
ax.set_ylabel("Emissions (kgCO2)");
plt.legend(['High Thermal Scenario','High Renewables Scenario'], loc='upper left');


In [ ]:
# ax.figure.savefig("images/BSS charging emissions aggregated for every 30 mins of day.png", bbox_inches='tight')

### Determine emissions for the shifted energy profile

#### Emissions for charging at BSSs ONLY

In [ ]:
df_merged_bms_gps_ch_energy_at_bss_30_mins = df_merged_bms_gps_ch_energy_at_bss_30_mins.reset_index()
df_merged_bms_gps_ch_energy_at_bss_30_mins['emissions_30_mins_hi_therm_coord'] = df_merged_bms_gps_ch_energy_at_bss_30_mins['tot_ch_energy_coord']/1000*df_hi_therm_AEF_30_mins['AEF_30_mins']
df_merged_bms_gps_ch_energy_at_bss_30_mins['emissions_30_mins_hi_renew_coord'] = df_merged_bms_gps_ch_energy_at_bss_30_mins['tot_ch_energy_coord']/1000*df_hi_renew_AEF_30_mins['AEF_30_mins']
df_merged_bms_gps_ch_energy_at_bss_30_mins.head()

In [ ]:
ax = df_merged_bms_gps_ch_energy_at_bss_30_mins.plot.bar(x='30_mins_inc', 
                                              y=['emissions_30_mins_hi_therm','emissions_30_mins_hi_renew','emissions_30_mins_hi_therm_coord', 'emissions_30_mins_hi_renew_coord'],
                                              ylim=[0,4],
                                              figsize=(12,4), 
                                              width=0.8,
                                              color=[ "dimgrey", "blue", "silver", "lightblue"],
                                              legend=True)
# ax.set_title('Charging emissions aggregated for every 30 mins of day');
ax.set_title('');
ax.set_xlabel("Time period (hours)");
ax.set_ylabel("Emissions (kgCO2)");
plt.legend(['High Thermal Scenario','High Renewables Scenario','High Thermal Co-ord Scenario','High Renewables Co-ord Scenario'], loc='upper left');


In [ ]:
# ax.figure.savefig("images/BSS charging emissions aggregated for every 30 mins of day (all scenarios).png", bbox_inches='tight')

#### Create bar chart to compare the total emissions for each scenario 

In [ ]:
df_emissions_total = df_merged_bms_gps_ch_energy_at_bss_30_mins[['emissions_30_mins_hi_therm','emissions_30_mins_hi_therm_coord','emissions_30_mins_hi_renew', 'emissions_30_mins_hi_renew_coord']].agg('sum').to_frame().transpose()
ax = df_emissions_total.plot.bar(legend=True, color=[ "dimgrey", "silver", "blue", "lightblue"])
# ax.set_title('Total charging emissions for each scenario');
ax.set_title('');
ax.set_xlabel("Scenario");
ax.set_ylabel("Total BSS emissions over analysis period (kgCO2)");
plt.xticks([])
plt.legend(['High Thermal Scenario','High Thermal Co-ord Scenario', 'High Renewables Scenario','High Renewables Co-ord Scenario'], bbox_to_anchor=(0.5, 1.3), loc='upper center');

In [ ]:
# ax.figure.savefig("images/Total BSS charging emissions over analysis period (all scenarios).png", bbox_inches='tight')

### Determine e-motorycle emissions factors (gCO2/km) based on charging emissions and distance driven

#### Input baseline emissions factor for ICE motorcycles:

In [ ]:
baseline_emissions_g_per_km = 69.1;

In [ ]:
# Estimate total distance driven using the charging energy from BSSs using the average energy intensity determined for the motorcycles
total_ch_energy_at_bss = df_merged_bms_gps_ch_energy_at_bss_30_mins['tot_ch_energy'].sum()
total_distance_from_bss_charging = total_ch_energy_at_bss/energy_intensity_avg
print('The total estimated distance driven using energy from BSS charging was: \n', 
      np.round(total_distance_from_bss_charging), ' km')

In [ ]:
# Determine emissions per km (gCO2/km) for each scenario
df_emissions_g_per_km = df_emissions_total/total_distance_from_bss_charging*1000
df_emissions_g_per_km

In [ ]:
df_emissions_g_per_km['baseline_ICE'] = baseline_emissions_g_per_km

In [ ]:
ax = df_emissions_g_per_km.plot.bar(legend=True, y=['baseline_ICE','emissions_30_mins_hi_therm','emissions_30_mins_hi_therm_coord','emissions_30_mins_hi_renew', 'emissions_30_mins_hi_renew_coord'], color=[ "red", "dimgrey", "silver", "blue", "lightblue"], width=1)
# ax.set_title('BSS e-motorcycles emissions factors (all scenarios)')
plt.xticks([])
plt.legend(['ICE Baseline','E2W High Thermal Scenario','E2W High Thermal Co-ord Scenario', 'E2W High Renewables Scenario','E2W High Renewables Co-ord Scenario'], bbox_to_anchor=(0.5, 1.35), loc='upper center');

In [ ]:
# ax.figure.savefig("images/BSS e-motorcycles emissions factors (all scenarios).png", bbox_inches='tight')

In [ ]:
df_emissions_g_per_km_reduction = (baseline_emissions_g_per_km - df_emissions_g_per_km)/baseline_emissions_g_per_km*100
df_emissions_g_per_km_reduction

### What amount of emissions were mitigated through shifting the evening peak charging energy?

In [ ]:
percentage_emissions_saved_coord_therm = (df_emissions_total['emissions_30_mins_hi_therm'] - df_emissions_total['emissions_30_mins_hi_therm_coord'])/df_emissions_total['emissions_30_mins_hi_therm']*100
percentage_emissions_saved_coord_renew = (df_emissions_total['emissions_30_mins_hi_renew'] - df_emissions_total['emissions_30_mins_hi_renew_coord'])/df_emissions_total['emissions_30_mins_hi_renew']*100
print('The total percentage change in emissions by shifting loads to off-peak periods was: \n', 
      np.round(percentage_emissions_saved_coord_therm.values[0], decimals = 2),'% for the high thermal scenario, and \n',
      np.round(percentage_emissions_saved_coord_renew.values[0], decimals = 2),'% for the high renewables scenario.')

## 9. Tariff analysis
### Load data
#### Set pathname for electricity tariff spreadsheet and then load as dataframe

In [ ]:
# Set pathname to file
pathname_elec_tariffs = ''

In [ ]:
# Import spreradsheet as dataframe
df_elec_tariffs = pd.read_excel(pathname_elec_tariffs, sheet_name='elec_cost', header=0)

# Display dataframe to inspect data
df_elec_tariffs.head()

### Calculate energy costs for the different scenarios for each hour

In [ ]:
# Create a dataframe with all the columns required to calculate the energy costs
df_energy_costs_hr = df_merged_bms_gps_ch_energy_at_bss_hr.reset_index().join(df_elec_tariffs[['S1_elec_cost_KES_kWh','S2_elec_cost_KES_kWh','S3_elec_cost_KES_kWh']])
df_energy_costs_hr['S1_total_cost'] = df_energy_costs_hr['tot_ch_energy']*df_energy_costs_hr['S1_elec_cost_KES_kWh']
df_energy_costs_hr['S2_total_cost'] = df_energy_costs_hr['tot_ch_energy']*df_energy_costs_hr['S2_elec_cost_KES_kWh']
df_energy_costs_hr['S3_total_cost'] = df_energy_costs_hr['tot_ch_energy']*df_energy_costs_hr['S3_elec_cost_KES_kWh']

# Display dataframe
df_energy_costs_hr.head()

### Create bar chart to compare the hourly energy costs for each different tariff scenario 

In [ ]:
ax = df_energy_costs_hr.plot.bar(x='hr_inc', 
                                              y=['S1_total_cost','S2_total_cost','S3_total_cost'], 
                                              figsize=(12,3), 
                                              width=0.8,
                                              color={'S1_total_cost': "darkblue", 'S2_total_cost': "blue", 'S3_total_cost':"lightblue"},
                                              legend=True)
# ax.set_title('Battery Swap Cabinet Charging Costs for every hour of day');
ax.set_title('');
ax.set_xlabel("Time period (hours)");
ax.set_ylabel("Electricity cost (KShs)");
plt.legend(['Domestic Tariff','E-mobility Tariff', 'E-mobility ToU Tariff']);

In [ ]:
# ax.figure.savefig("images/Charging Costs for every hour of day.png", bbox_inches='tight')

### Determine energy costs for the shifted energy profile with the "Scenario 3" ToU tariff structure

In [ ]:
df_energy_costs_hr['tot_ch_energy_coord'] = df_merged_bms_gps_ch_energy_at_bss_hr.reset_index().tot_ch_energy_coord
df_energy_costs_hr['S3_total_cost_coord'] = df_energy_costs_hr['S3_elec_cost_KES_kWh']*df_energy_costs_hr['tot_ch_energy_coord']

# Display the dataframe to inpect results
df_energy_costs_hr.head()

### Create bar chart to compare the hourly energy costs for each different tariff scenario 

In [ ]:
ax = df_energy_costs_hr.plot.bar(x='hr_inc', 
                                              y=['S1_total_cost','S2_total_cost','S3_total_cost', 'S3_total_cost_coord'], 
                                              figsize=(12,4), 
                                              width=0.8,
                                              color={'S1_total_cost': "darkblue", 'S2_total_cost': "blue", 'S3_total_cost':"lightblue", 'S3_total_cost_coord':'darkseagreen'},
                                              legend=True)
# ax.set_title('BSS charging costs aggregated for every hour of day (all tariff scenarios)');
ax.set_title('');
ax.set_xlabel("Time (hours)");
ax.set_ylabel("Electricity cost (KShs)");
plt.legend(['Domestic Tariff','E-mobility Tariff', 'E-mobility ToU Tariff', 'E-mobility ToU Tariff with co-ordinated charging']);

In [ ]:
# ax.figure.savefig("images/BSS charging costs aggregated for every hour of day (all tariff scenarios)'.png", bbox_inches='tight')

### Create bar chart to compare the total costs for each different tariff scenario 

In [ ]:
df_energy_costs_total = df_energy_costs_hr[['S1_total_cost','S2_total_cost','S3_total_cost','S3_total_cost_coord']].agg('sum').to_frame().transpose()
ax = df_energy_costs_total.plot.bar(legend=True, width=1, color=["darkblue","blue", "lightblue", 'darkseagreen'])
# ax.set_title('BSS total charging costs over analysis period (all tariff scenarios)');
ax.set_title('');
ax.set_xlabel('Scenario');
ax.set_ylabel("Total electricity cost over analysis period (KShs)");
plt.xticks([]);
plt.legend(['Domestic Tariff','E-mobility Tariff', 'E-mobility ToU Tariff', 'E-mobility ToU Tariff with co-ordinated charging'], bbox_to_anchor=(0.5, 1.3), loc='upper center');

In [ ]:
# ax.figure.savefig("images/BSS total charging costs over analysis period (all tariff scenarios)'.png", bbox_inches='tight')

In [ ]:
df_charging_cost_reduction = (df_energy_costs_total['S1_total_cost'].values[0] - df_energy_costs_total)/df_energy_costs_total*100
df_charging_cost_reduction

### Determine averaged costs

In [ ]:
total_ch_energy_at_bss = df_merged_bms_gps_ch_energy_at_bss_30_mins['tot_ch_energy'].sum()
df_energy_costs_average_per_kwh = df_energy_costs_total/total_ch_energy_at_bss
df_energy_costs_average_per_kwh

In [ ]:
ax = df_energy_costs_average_per_kwh.plot.bar(legend=True, width=1, color=["darkblue","blue", "lightblue", 'darkseagreen'])
# ax.set_title('Average electricity costs for BSS (all tariff scenarios)');
ax.set_title('');
ax.set_xlabel('Scenario');
ax.set_ylabel("Average electricity cost (KShs/kWh)");
plt.xticks([]);
plt.legend(['Domestic Tariff','E-mobility Tariff', 'E-mobility ToU Tariff', 'E-mobility ToU Tariff with co-ordinated charging'], bbox_to_anchor=(0.5, 1.3), loc='upper center');

In [ ]:
# ax.figure.savefig("images/Average electricity costs for BSS (all tariff scenarios)'.png", bbox_inches='tight')

### How much was saved using the ToU tariff and shifting the evening peak charging energy?

In [ ]:
percentage_saved_shift = (df_energy_costs_hr['S2_total_cost'].sum() - df_energy_costs_hr['S3_total_cost_coord'].sum())/df_energy_costs_hr['S2_total_cost'].sum()*100
percentage_saved_S3vS2 = (df_energy_costs_hr['S2_total_cost'].sum() - df_energy_costs_hr['S3_total_cost'].sum())/df_energy_costs_hr['S2_total_cost'].sum()*100
print('The total percentage of electricity costs saved by shifting loads to off-peak periods using the E-mobility ToU \ntariffs was ', 
      np.round(percentage_saved_shift, decimals = 2),'% compared to a savings of ', np.round(percentage_saved_S3vS2, decimals = 2),'% using E-mobility ToU tariffs without shifting load.')